# Исследование надежности заемщиков


## Открытие данных

Импортируем библиотеку `pandas`. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`:

In [ ]:
import pandas as pd # импортируйт библиотеки pandas
import numpy as np

pd.options.mode.chained_assignment = None #предотвращение появление сообщения SettingWithCopyWarning

Прочитаем файлы из папки `/datasets`.

In [ ]:
 # чтение csv-файла
path = '/datasets/'
web_path = 'https://****'
data = 'data.csv'

try:
    data_raw = pd.read_csv(path + data)
except:
    data_raw = pd.read_csv(web_path + data)

Выведим первые 20 строчек датафрейма `data` на экран.

In [ ]:
data_raw.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [ ]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в столбце `education` имееются значения записанные в разной литерации — расхождения с хорошим стилем.

## Предобработка данных

Проведем стандартную предобработку, включающаю проверку явных дубликатов, нулевых и пропущенных значений, а также изменим тип данных и преобразуем категориальные переменные.

In [ ]:
data = data_raw.copy()

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.

In [ ]:
pd.concat([data.isna().sum(), data.isna().mean()],
          axis=1, keys=['sum', 'conversation']).style.format({'conversation': '{:.2%}'}).background_gradient('coolwarm') # подсчёт пропусков

,sum,conversation
children,0,0.00%
days_employed,2174,10.10%
dob_years,0,0.00%
education,0,0.00%
education_id,0,0.00%
family_status,0,0.00%
family_status_id,0,0.00%
gender,0,0.00%
income_type,0,0.00%
debt,0,0.00%


В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [ ]:
for i in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (i) и есть пропуски в total_income
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == i), 'total_income'].median()

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим ещё раз и выведите количество пропущенных значений для каждого столбца с помощью двух методов.

In [ ]:
pd.concat([data.isna().sum(), data.isna().mean()],
          axis=1, keys=['sum', 'conversation']).style.format({'conversation': '{:.2%}'}).background_gradient('coolwarm') # подсчёт пропусков

,sum,conversation
children,0,0.00%
days_employed,0,0.00%
dob_years,0,0.00%
education,0,0.00%
education_id,0,0.00%
family_status,0,0.00%
family_status_id,0,0.00%
gender,0,0.00%
income_type,0,0.00%
debt,0,0.00%


Возможные причины появления пропусков в исходных данных могли послужить: человеческий фактор, ошибки системы или процесса выгрузки.

Заполнить пропуски медианным значением — лучшее решение для количественных переменных, т.к. реднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.

### Обработка аномальных значений

Исследуем данные, используя функцию `describe`.

In [ ]:
data.describe(include=[np.number]).transpose()

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,21525.0,63550.497071,141150.420058,-18388.949901,-2570.047544,-1355.683356,-316.240646,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,21525.0,165225.324514,98043.665491,20667.263793,107798.172619,142594.396847,195549.940861,2.265604e+06


В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [ ]:
data['education'] = data['education'].str.lower()

Проверим остальные столбцы, для этого выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, то удалим их.

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Проверка гипотез

Исследуем данные и ответим на вопросы.

#### Зависимость между количеством детей

**Гипотеза:**

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
#выведем сводную таблицу по столбцу children
children_debt = data.pivot_table(index='children', values='debt', aggfunc=['count','sum'])
children_debt['conversion_debt'] = (children_debt[('sum', 'debt')]/children_debt[('count', 'debt')])*100
children_debt

,count,sum,conversion_debt
,debt,debt,
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


**Вывод:** Прямой зависимости как таковой нет, однако, исходя из таблицы видно, что реже всех не возвращают кредиты в срок клиенты без детей - 7,54%, доля просроченных кредитов у клиентов имеющих от 1 до 4 детей варьируется от 8,18 % до 9,76 %. Выборка с клиентами, имеющими 5 детей, является непоказательной из-за малого количества клиентов, т.к. поэтому ее можно не учитывать.

#### Зависимость между семейным положением

**Гипотеза:**

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#создадим сводную таблицу по столбцу family_status
family_debt = data.pivot_table(index='family_status', values='debt', aggfunc=['count','sum'])
family_debt['conversion_debt'] = (family_debt[('sum', 'debt')]/family_debt[('count', 'debt')])*100
family_debt

,count,sum,conversion_debt
,debt,debt,
family_status,,,
Не женат / не замужем,2796,273,9.763948
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558


**Вывод:** В большей степени просрочку допускают холостые заемщики - 9,76 % от общего количества всех не женатых / не замужних, на общем фоне также выделяются лица, живущие в гражданском браке - 9,31 % от общего числа состоящих в гражданском браке.

#### Зависимость между уровнем дохода

**Гипотеза:**

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#создадим сводную таблицу по столбцу total_income_category
total_income_debt = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count','sum'])
total_income_debt['conversion_debt'] = (total_income_debt[('sum', 'debt')]/total_income_debt[('count', 'debt')])*100
total_income_debt

,count,sum,conversion_debt
,debt,debt,
total_income_category,,,
A,25,2,8.000000
B,5014,354,7.060231
C,15921,1353,8.498210
D,349,21,6.017192
E,22,2,9.090909


**Вывод:** Прямая зависимость между уровнем дохода и неуплатой кредита в срок наблюдается только между категориями B (200001–1000000 руб.) - 7,06% и категорией дохода C (50001–200000 руб.) - 8,49%. Другие категории доходов мы не можем учитывать, т.к. выборки по категориям A, D и E недостаточно. Следовательно, чаще возращают кредит клиенты с более высоким уровнем дохода.

#### Зависимость от цели кредита

**Гипотеза:**

Оказывают ли влияние разные цели кредита на его возврат в срок?

In [ ]:
#создадим сводную таблицу по столбцу purpose_category
purpose_debt = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count','sum'])
purpose_debt['conversion_debt'] = (purpose_debt[('sum', 'debt')]/purpose_debt[('count', 'debt')])*100
purpose_debt

,count,sum,conversion_debt
,debt,debt,
purpose_category,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


**Вывод:** Операции с автомобилем имеют самую большую долю невозвратов в срок - 9,35%, наименьшую же имеют операции с недвижимостью - 7,25%.

## Общий вывод

В ходе исследования подтвердилось лишь две зависимости:
1. Клиенты, которые официально состоят в браке или состояли чаще всего погашают кредит в срок, особенно чаще в срок платят овдовевшие или разведенные клиенты.
2. Клиенты с более высоким доходом реже допускают просрочку по кредиту.

Помимо этого частично подтвердилась зависимость с количеством детей, но она лишь показывает, что бездетные клиенты лучше всего справляются с обязанностями по кредиту. Также лучше всего выдавать кредиты на покупки, связанные с операциями с недвижимостью или для проведения свадеб, т.к. по ним невозврат в срок гораздо меньше.

Подведем итоги и составим образ потенциальных клиентов, способных погасить кредит в срок в рамках модели кредитного скоринга:
1. Бездетные.
2. Состоят или состояли в браке, но не  в гражданском.
3. Имеют доход категории B или выше.
4. Целью кредита являются операции с недвижимостью или проведение свадьбы.

В отношении следующих клиентов стоит пересмотреть скоринговую политику:
1. Клиентов с детьми.
2. Не женатых или холостых, а также клиентов, состоящих в гражданском браке.
3. Клиентов, имеющих доход ниже категории B.
4. Кредитов, целями которых выступают операции с автомобилем или получением образования.